In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib as plt
from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI()
pd.options.display.float_format = '{:.2f}'.format 

df = pd.read_json(r'dataset\response_1692132599095.json', dtype='unicode')

In [ ]:
#testeo si funciona la conexion con la api
cg.ping()

In [ ]:
df.columns

In [ ]:
crypto_lista = cg.get_coins_list()
df_cryptos= pd.DataFrame.from_dict(crypto_lista)

df_cryptos

In [ ]:
cryptos_categorias = pd.DataFrame(cg.get_coins_categories_list())
cryptos_categorias

In [ ]:

#considero que no voy a utilizar estas columnas:
df.drop(columns=['image','last_updated', 'roi', 'ath_date','atl_date',  'high_24h',
                 'low_24h','price_change_24h','price_change_percentage_24h','market_cap_change_24h',
                 'market_cap_change_percentage_24h'], inplace=True)

In [ ]:
coins_lista=df["id"].tolist()



In [ ]:
#con esta funcion obtengo una lista con el precio de una crypto por dia en el lapso de un año
#la idea seria ver que tanto varia el precio a lo largo de un año y de eso armar un numero que signifique la volatilidad
#de esa crypto moneda como una variable importante para determinar que cryptomonedas son mas estables que otras

def get_df_volatility_by_crypto(coins_lista):
    
    volatility_by_crypto = []

    #obtengo precio por dia de cada crypto durante un año
    for coin_id in coins_lista:

        price_per_day=[]
        
        #llamo la funcion de coingecko para obtener datos de la crypto
        row=cg.get_coin_market_chart_by_id(id= coin_id, vs_currency = 'usd', days ='365', interval='daily', precision='5')
        
        #solo obtengo los precios diarios de la crypto
        for i in range(len(row['prices'])):
            price_per_day.append(row['prices'][i][1])
            
            
        #calculo la volatilidad de la crypto en relacion a su desviacion estandard
        volatility=np.std(price_per_day)
        volatility_by_crypto.append({
            "id":coin_id,
            "volatilidad":volatility 
        })

    volatility_by_crypto=pd.DataFrame(volatility_by_crypto)

    #estandarizo cada valor para que tenga sentido hacer comparaciones, ya que cada valor sin ser estandarizado esta afectado 
    #por la diferencia de precios entre cada crypto
    volatility_by_crypto["volatilidad_estandarizada"] = (volatility_by_crypto["volatilidad"] - volatility_by_crypto["volatilidad"].mean()) / volatility_by_crypto["volatilidad"].std()
    return volatility_by_crypto



In [ ]:
result=get_df_volatility_by_crypto(coins_lista)
result

In [18]:
result.sort_values(by="volatilidad_estandarizada", ascending=False)

,id,volatilidad,volatilidad_estandarizada
16,wrapped-bitcoin,4789.57,6.95
0,bitcoin,4780.44,6.94
43,rocket-pool-eth,278.90,0.25
6,staked-ether,256.86,0.21
1,ethereum,252.51,0.21
...,...,...,...
95,terra-luna,0.00,-0.17
72,ecash,0.00,-0.17
13,shiba-inu,0.00,-0.17
93,bittorrent,0.00,-0.17


# Clasificación de las variables

### cuantitativos
* discretos = enteros
* continuos = se divide infinitamente

### cualitativos
* categoricos = de una categoria o clase especifica
* ordinales = de categoria en la que IMPORTA un orden o jerarquía

### Categoricas 

* id
* symbol
* name

### Discretas
* Ninguna

### Continuas  

* current_price 
* market_cap 
* fully_diluted_valuation --> si todas las monedas de una crypto estuvieran en circulacion, es para saber el potencial de una moneda con su valor actual
* total_volume 
* high_24h 
* low_24h
* circulating_supply
* total_supply
* max_supply
* ath
* ath_change_percentage
* atl
* atl_change_percentage
* ath_date
* atl_date
* roi
* price_change_24h
* price_change_percentage_24h
* market_cap_change_24h
* market_cap_change_percentage_24h

### Ordinales 

* market_cap_rank

## Conclusión

No hay variables discretas en mi opinion ya que si bien variables como circulating supply usualmente se refiere a una cantidad espcífica de monedas en circulación, cualquier moneda puede ser separada en partes más pequeñas de forma infinita, hence, es una variable continua.

La gran mayoría de variables son continuas ya que se trata de dinero o cryptos que, como ya dije, pueden dividirse infinitamente.

Cabe recalcar la importancia de la variable ordinal "market_cap_rank" ya que juega un papel muy importante en el mundo de las cryptos aquellas rankeadas en lo alto. Esto se debe a que ganan mayor fama y reconocimiento. Esta fama es incluso mayor si se trata de una crypto que logra mantenerse durante largos periodos de tiempo (años) en dicho ranking. Por ejemplo bitcoin, ethereum, usdt, etc.




In [ ]:

df.columns.tolist()

In [ ]:
#considero que para poder continuar, debo poner el tipo de dato que corresponde a cada columna

strings= ["id", "symbol", "name"]
floats=['current_price',
 'market_cap',
 'fully_diluted_valuation',
 'total_volume',
 'circulating_supply',
 'total_supply',
 'max_supply',
 'ath',
 'ath_change_percentage',
 'atl',
 'atl_change_percentage']

for i in strings:
    df[i].fillna(0)
    df[i] = df[i].astype("string")

for i in floats:
    df[i].replace('None', pd.NA, inplace=True)
    df[i].fillna(0, inplace=True)
    df[i] = df[i].astype("float")

df["market_cap_rank"] = df["market_cap_rank"].astype("int")

df.dtypes

# Como quiero plantear este EDA

Mi objetivo en este EDA es obtener insights sobre las 10 mejores criptomonedas como potencial inversion para clientes de una empresa.

Este analisis exploratorio de los datos va a estar dividido en dos partes:

* Análisis de Bitcoin y sus altcoins (altcoins = aquellas criptomonedas que no son Bitcoin)

* Análisis de stable coins (aquellas criptomonedas que tienen su valor anclado al dólar)

Este acercamiento se debe a que por un lado el comportamiento de las altcoins es muy diferente al de las stablecoins. Lo que quiero lograr es marcar una diferencia entre aquellas criptos que son más riesgosas pero mucho más volátiles y aquellas que son estables pero que no van a suponer un retorno alto.

In [ ]:
#divido mi dataframe en stablecoins y criptomonedas

df_stablecoins = df[(df["current_price"] > 0.99) & (df["current_price"] < 1)]

df_altcoins_and_btc = df[~((df["current_price"] >= 0.99) & (df["current_price"] <= 1))] #usar ~ para negar una condicion 
                                                                                #se hace con Alt Gr + (boton de asterisco al lado del enter)


df_altcoins_and_btc


In [ ]:
df.describe()
#busco informacón relevante que me ayude a comenzar a filtrar ciertas criptomonedas

In [ ]:
#df.drop(df[df['name'] == 'Bitcoin'].index, inplace=True)
#df.drop(df[df['name'] == 'Ethereum'].index, inplace=True)


#Plot histograms of numerical data
%matplotlib inline
import matplotlib.pyplot as plt
df.hist(bins = 50, figsize = (20,15))
plt.show